<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 3: Web APIs &NLP

### Introduction

The AMD strategic PLanning (consumer GPUs) team wants our team to find out whether the 'negative stigma' associated with AMD steams from genuine issues of it is a simply matter of perception


### Background 

Nvidia's product were well known for graphic processing solutions and it were common used for AI and deep learning systems. 

AMD produces a wide range of microchip products. AMD also well known for graphic processors, system processors, RAM memory and hard drives. However, AMD was often regarded as a budget brand although thier product were still closly match up with Nvidia high end products. 

Depsite constant effort that AMD has put in to improve, market still perceive AMD was a weaker brand as compared to Nvidia. On top of that, AMD user has been constantly complain about the driver issues of AMD.


### Problem Statement

It is unclear whether the perception of AMD product being problematic is a marketing or an engineering issue? If this negative stigma continues to drag for AMD, it will definitely affect the bottom line of business to the point that the company unable to make a profit.   


### Approaches that we going to take

In order to gain more insight, we are going to scrap the posts and comments from Reddit for both AMD and Nvidia. For both AMD and Nvidia subreddits, negative posts such as post with cursing lauguage without specific reference to any of the problem, the post will be removed. Therefore, we would only expect there to be negative posts about the actual problems. 

With this mechanism in place and using sentiment analysis, if it is an engineering issue, we would expect to see many more negative posts on r/amd than r/nvidia. If the negative posts are similar, then it is likely a marketing issue. 

### Objectives

- We want to build a model that has the ability to differentiate between comments about AMD and nvidia for widerspread deployment e.g PC forum (in the event that becomes necessary) 

- Using sentiment analysis on the same data set, the result could be used to determine whether is it a marketing issue or an engineering issue? so AMD can deploy resources to improve the perception of the product

### Data

- Titles, body text, comments posted to r/amd and r/nvidia

### Methods and Tool: 

1. TfidfVectorizer 
2. CountVectorizer 
3. Naive Bayes 
4. Random Forest 
5. Logistic Regression 
6. RidgeClassifier

### Success Metrics

1. F1 score for classification, we want to minimise both false positive and false negatives. This will be used to select the classification model for deployment on GPU discussion forums. 

2. Sentiment Analysis scores, will use this score to recommend, to either deploy more resources to marketing or engineering. 

# 1.0 Scraper  Mechanism

In [2]:
# lib
import praw 
import time
import json
import pickle
import requests
import logging

### 1.1.0 Initialize PRAW

- to get the client_id and client_secret can use the link below 
- https://www.reddit.com/prefs/apps/

In [3]:
reddit = praw.Reddit(client_id="0z7Yqipy8rSrEyNrvqy3YQ", 
                     client_secret="F4LVjgGlfKqRB8Zk0FxF5-8FcbOSdg",
                    user_agent="python:my_reddit_scraper:v1.0 (by /u/MKLim182)",
                    username='MKLIM182',
                    password='Yellowcard182')

In [15]:
# create a subreddit object for Amd/nvidia
subreddit = reddit.subreddit('nvidia')


### 1.2.0 Load from the previous file (where I last left off from scraping) and retrieve the last_post_id 

Using the function below, to retrieve out the last post id where I left off from. 

In [16]:
def get_last_post_id_from_jsonl(filename):
    last_line = None
    with open(filename, 'r') as f:
        for line in f:
            last_line = line
    if last_line:
        last_post_data = json.loads(last_line)
        return last_post_data.get('Post ID')
    return None

# Use the function to get the last post ID
last_post_id = get_last_post_id_from_jsonl('nvidia_posts.jsonl')
print(f"Last post ID: {last_post_id}")

Last post ID: 1645wyh


### 1.3.0 Scraping Mechanism

Using the function below, for each request it will only scrap 600 posts, after scraping 600 posts it will save in both formate Json or Pickle. 

while scraping from reddit, it will only scrap the informations that I wanted in the extract_post_data functions as shown below. It will also scrap every single comment from each post as shown in extract_comment_data. 

We can set the number of post that we wanted to scrap at "post_count", after scraping each post the program will rest for 1sec then it will continue the next scrap. 

Once it has scrap 600 posts, the program will save all the posts to Josn and Pickle and sleep for 10sec, before it starting to scrap again.


In [17]:
# Initialize an empty list to store posts
posts = []

# Counter for the number of posts scraped
post_count = 0

# File to save scraped posts in JSON format 
output_file_json  = 'nvidia_posts.jsonl'

# File to save scraped posts in pickle format
output_file_pickle = "nvidia_posts.pkl"

# Set fixed limit for each request
request_limit = 600

# Initialize the last post ID outside the loop
last_post_id = None 

MAX_RETRIES = 2  # Maximum number of retries for a request

def extract_post_data(post):
    """Extract data from a post."""
    return {
        'Score': post.score,
        'Post ID': post.id,
        'Creation Time': post.created_utc,
        'Awards': post.total_awards_received,
        'Subreddit': str(post.subreddit),
        'Gilded': post.gilded,
        'Award Details': [award['name'] for award in post.all_awardings],  # Assuming 'name' is the attribute for award name
        'date': post.created_utc,
        'title': post.title,
        'is_self': post.is_self,
        'selftext': post.selftext,
        'upvotes': post.score,
        'downvotes': post.downs,
        'upvote_ratio': post.upvote_ratio,
        'n_comments': post.num_comments,
        'author': str(post.author),
        'distinguished': post.distinguished,
        'Flair': post.link_flair_text
    }

def extract_comments_data(post, post_data):
    """Extract data from comments associated with a post."""
    comments_data = []
    post.comments.replace_more(limit=None)
    for comment in post.comments.list():
        comment_data = post_data.copy()
        comment_data['comment'] = comment.body
        comments_data.append(comment_data)
    return comments_data

def handle_429(response):
    """Handle 429 errors by sleeping for the duration specified in Retry-After or default to 5 minutes."""
    retry_after = response.headers.get('Retry-After')
    if retry_after:
        sleep_duration = int(retry_after)
    else:
        sleep_duration = 300  # Default to 5 minutes
    print(f"429 error detected. Sleeping for {sleep_duration} seconds.")
    time.sleep(sleep_duration)
    

start_time = time.time()  # Start the timer

try: 
    # Continue scrap until not more than 1200 post 
    while post_count < 1200: 
        retries = 0
        while retries < MAX_RETRIES:
            try:
                continued_posts = list(subreddit.new(limit=request_limit, 
                                                     params={"after": last_post_id} if last_post_id else {}))
                break  # If successful, break out of the retry loop
            except requests.exceptions.HTTPError as e:
                if e.response.status_code == 429:
                    retry_after = e.response.headers.get('Retry-After', 300)  # Default to 5 minutes if not provided
                    print(f"429 error detected. Sleeping for {retry_after} seconds.")
                    time.sleep(int(retry_after))
                    retries += 1
                else:
                    raise  # If it's not a 429 error, raise it

        if retries == MAX_RETRIES:
            print("Max retries reached. Exiting.")
            break

        if not continued_posts:
            print("No more new posts found.")
            break

        for post in continued_posts:
            post_data = extract_post_data(post)
            comments_data = extract_comments_data(post, post_data)
            
            with open(output_file_json, 'a') as f:
                f.write(json.dumps(post_data))
                f.write('\n')
                for comment_data in comments_data:
                    f.write(json.dumps(comment_data))
                    f.write('\n')
            
            with open(output_file_pickle, 'ab') as f:
                pickle.dump(post_data, f)
                for comment_data in comments_data:
                    pickle.dump(comment_data, f)

            post_count += 1
            print(f"Scraped {post_count} posts so far...")
            time.sleep(1)

        last_post_id = post.id
        time.sleep(10)

except Exception as e:
    print(f"An error occurred: {e}")

end_time = time.time()  # End the timer

print("Scraping completed!")
print(f"Total time taken for scraping: {end_time - start_time} seconds")         

Scraped 1 posts so far...
Scraped 2 posts so far...
Scraped 3 posts so far...
Scraped 4 posts so far...
Scraped 5 posts so far...
Scraped 6 posts so far...
Scraped 7 posts so far...
Scraped 8 posts so far...
Scraped 9 posts so far...
Scraped 10 posts so far...
Scraped 11 posts so far...
Scraped 12 posts so far...
Scraped 13 posts so far...
Scraped 14 posts so far...
Scraped 15 posts so far...
Scraped 16 posts so far...
Scraped 17 posts so far...
Scraped 18 posts so far...
Scraped 19 posts so far...
Scraped 20 posts so far...
Scraped 21 posts so far...
Scraped 22 posts so far...
Scraped 23 posts so far...
Scraped 24 posts so far...
Scraped 25 posts so far...
Scraped 26 posts so far...
Scraped 27 posts so far...
Scraped 28 posts so far...
Scraped 29 posts so far...
Scraped 30 posts so far...
Scraped 31 posts so far...
Scraped 32 posts so far...
Scraped 33 posts so far...
Scraped 34 posts so far...
Scraped 35 posts so far...
Scraped 36 posts so far...
Scraped 37 posts so far...
Scraped 38

Scraped 298 posts so far...
Scraped 299 posts so far...
Scraped 300 posts so far...
Scraped 301 posts so far...
Scraped 302 posts so far...
Scraped 303 posts so far...
Scraped 304 posts so far...
Scraped 305 posts so far...
Scraped 306 posts so far...
Scraped 307 posts so far...
Scraped 308 posts so far...
Scraped 309 posts so far...
Scraped 310 posts so far...
Scraped 311 posts so far...
Scraped 312 posts so far...
Scraped 313 posts so far...
Scraped 314 posts so far...
Scraped 315 posts so far...
Scraped 316 posts so far...
Scraped 317 posts so far...
Scraped 318 posts so far...
Scraped 319 posts so far...
Scraped 320 posts so far...
Scraped 321 posts so far...
Scraped 322 posts so far...
Scraped 323 posts so far...
Scraped 324 posts so far...
Scraped 325 posts so far...
Scraped 326 posts so far...
Scraped 327 posts so far...
Scraped 328 posts so far...
Scraped 329 posts so far...
Scraped 330 posts so far...
Scraped 331 posts so far...
Scraped 332 posts so far...
Scraped 333 posts so

Scraped 591 posts so far...
Scraped 592 posts so far...
Scraped 593 posts so far...
Scraped 594 posts so far...
Scraped 595 posts so far...
Scraped 596 posts so far...
Scraped 597 posts so far...
Scraped 598 posts so far...
Scraped 599 posts so far...
Scraped 600 posts so far...
Scraped 601 posts so far...
Scraped 602 posts so far...
Scraped 603 posts so far...
Scraped 604 posts so far...
Scraped 605 posts so far...
Scraped 606 posts so far...
Scraped 607 posts so far...
Scraped 608 posts so far...
Scraped 609 posts so far...
Scraped 610 posts so far...
Scraped 611 posts so far...
Scraped 612 posts so far...
Scraped 613 posts so far...
Scraped 614 posts so far...
Scraped 615 posts so far...
Scraped 616 posts so far...
Scraped 617 posts so far...
Scraped 618 posts so far...
Scraped 619 posts so far...
Scraped 620 posts so far...
Scraped 621 posts so far...
Scraped 622 posts so far...
Scraped 623 posts so far...
Scraped 624 posts so far...
Scraped 625 posts so far...
Scraped 626 posts so

Scraped 884 posts so far...
Scraped 885 posts so far...
Scraped 886 posts so far...
Scraped 887 posts so far...
Scraped 888 posts so far...
Scraped 889 posts so far...
Scraped 890 posts so far...
Scraped 891 posts so far...
Scraped 892 posts so far...
Scraped 893 posts so far...
Scraped 894 posts so far...
Scraped 895 posts so far...
Scraped 896 posts so far...
Scraped 897 posts so far...
Scraped 898 posts so far...
Scraped 899 posts so far...
Scraped 900 posts so far...
Scraped 901 posts so far...
Scraped 902 posts so far...
Scraped 903 posts so far...
Scraped 904 posts so far...
Scraped 905 posts so far...
Scraped 906 posts so far...
Scraped 907 posts so far...
Scraped 908 posts so far...
Scraped 909 posts so far...
Scraped 910 posts so far...
Scraped 911 posts so far...
Scraped 912 posts so far...
Scraped 913 posts so far...
Scraped 914 posts so far...
Scraped 915 posts so far...
Scraped 916 posts so far...
Scraped 917 posts so far...
Scraped 918 posts so far...
Scraped 919 posts so

Scraped 1171 posts so far...
Scraped 1172 posts so far...
Scraped 1173 posts so far...
Scraped 1174 posts so far...
Scraped 1175 posts so far...
Scraped 1176 posts so far...
Scraped 1177 posts so far...
Scraped 1178 posts so far...
Scraped 1179 posts so far...
Scraped 1180 posts so far...
Scraped 1181 posts so far...
Scraped 1182 posts so far...
Scraped 1183 posts so far...
Scraped 1184 posts so far...
Scraped 1185 posts so far...
Scraped 1186 posts so far...
Scraped 1187 posts so far...
Scraped 1188 posts so far...
Scraped 1189 posts so far...
Scraped 1190 posts so far...
Scraped 1191 posts so far...
Scraped 1192 posts so far...
Scraped 1193 posts so far...
Scraped 1194 posts so far...
Scraped 1195 posts so far...
Scraped 1196 posts so far...
Scraped 1197 posts so far...
Scraped 1198 posts so far...
Scraped 1199 posts so far...
Scraped 1200 posts so far...
Scraping completed!
Total time taken for scraping: 2970.083225250244 seconds
